In [1]:
import asyncio
import aiohttp
import time

from bs4 import BeautifulSoup
import pandas as pd

In [4]:
hatali_islemler = []
tum_doktorlar = []

async def doktor_profil_ozetini_getir(session, sayfa_no):
    url = "https://www.doktorsitesi.com/tumuzmanlar?sayfa=" + str(sayfa_no)
    try:
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')
            az_content = soup.find('div', class_='az-content')
            az_main_wrappers = az_content.find_all('div', class_='az-main-wrapper')
            doktorlar = []
            for az_main_wrapper in az_main_wrappers:
                verified = az_main_wrapper.find('div', class_='verified')
                dogrulanmis_profil = False
                if verified is not None:
                    dogrulanmis_profil = verified.text
                resim = az_main_wrapper.find('img')
                resim_linki = resim['src']
                cinsiyet = resim['data-gender']
                profil = az_main_wrapper.find('a')
                # <a href="https://www.doktorsitesi.com/ahmet-dincer/psikoloji/istanbul"> <span>Psk.</span> Ahmet Dinçer </a>        
                profil_linki = profil['href']
                konum = profil_linki.split('/')[-1]
                uzmanlik_alani = profil_linki.split('/')[-2]
                # unvanı <span>Psk.</span> olarak alır
                unvan = profil.find('span').text
                # ismi buradan alıp ve temizler \nPsk.\n                                Arzu Kantarcıoğlu\n
                isim = profil.text.split('\n')[2].strip()

                doktorlar.append({
                    'resim_linki': resim_linki,
                    'unvan': unvan,
                    'isim': isim,
                    'dogrulanmis_profil': dogrulanmis_profil,
                    'profil_linki': profil_linki,
                    'cinsiyet': cinsiyet,
                    'konum': konum,
                    'uzmanlik_alani': uzmanlik_alani
                })
            tum_doktorlar.extend(doktorlar)
    except Exception as e:
        print(f"Error: {e}")
        hatali_islemler.append(sayfa_no)

In [5]:
baslama_zamani = time.time()
async with aiohttp.ClientSession() as session:
    gorevler = []
    for i in range(1, 999):
        gorevler.append(doktor_profil_ozetini_getir(session, i))
    await asyncio.gather(*gorevler)
bitis_zamani = time.time()

print(f"Toplam {len(tum_doktorlar)} doktor profili çekildi. Hatalı işlem sayısı: {len(hatali_islemler)} Toplam süre: {bitis_zamani - baslama_zamani} saniye")

Toplam 19957 doktor profili çekildi. Hatalı işlem sayısı: 0 Toplam süre: 55.3374080657959 saniye


In [7]:
tum_doktorlar_df = pd.DataFrame(tum_doktorlar)
tum_doktorlar_df.to_csv('tum_doktorlar.csv', index=False)
tum_doktorlar_df.head()

,resim_linki,unvan,isim,dogrulanmis_profil,profil_linki,cinsiyet,konum,uzmanlik_alani
0,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,A. Gökçen Erdoğan,,https://www.doktorsitesi.com/op-dr-a-gokcen-er...,female,ankara,kadin-hastaliklari-ve-dogum
1,https://www.doktorsitesi.com/media/cache/profi...,Psk.,Aysel Ülgüner,,https://www.doktorsitesi.com/psk-aysel-ulguner...,female,izmir,psikoloji
2,https://www.doktorsitesi.com/media/cache/profi...,Dyt.,Aynur Mertoğlu,,https://www.doktorsitesi.com/dyt-aynur-mertogl...,female,zonguldak,diyetisyen
3,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Akin Savci,,https://www.doktorsitesi.com/op-dr-akin-savci/...,male,sakarya,kadin-hastaliklari-ve-dogum
4,https://www.doktorsitesi.com/media/cache/profi...,Dt.,Aysun Doğanay,,https://www.doktorsitesi.com/dt-aysun-doganay/...,female,istanbul,dis-hekimi-ortodonti
